In [ ]:
def y_bez1(x):
    #ybez1
    y11 = x * 2
    y12 = x + 3

    return y11, y12

def y_bez2(x):
    #ybez2
    y21 = x ** 2
    y22 = x - 1

    return y21, y22

roots = [1, 2, 3, 4]  # Lista de elementos

intersections = []  # Lista para almacenar las intersecciones

for root in roots:
    y11, y12 = y_bez1(root)
    y21, y22 = y_bez2(root)
    
    if y11 in (y21, y22):
        intersections.append((root, y11))
    if y12 in (y21, y22):
        intersections.append((root, y12))

print(intersections)


In [1]:
from svgpathtools import Path, Line, QuadraticBezier, CubicBezier, Arc, disvg
import numpy as np
seg1 = QuadraticBezier(250+100j, 200+500j, 300+100j)  # A cubic beginning at (300, 100) and ending at (200, 300)
seg2 = QuadraticBezier(400+150j, 0+100j, 400+200j)  # A line beginning at (200, 300) and ending at (250, 350)
path = Path(seg1, seg2)

In [4]:
x01, x11, x21 = seg1.start.real, seg1.control.real, seg1.end.real
x02, x12, x22 = seg2.start.real, seg2.control.real, seg2.end.real
y01, y11, y21 = seg1.start.imag, seg1.control.imag, seg1.end.imag
y02, y12, y22 = seg2.start.imag, seg2.control.imag, seg2.end.imag

A1, B1 = x01 - 2*x11 + x21, 2*x11 - 2*x01
A2, B2 = x02 - 2*x12 + x22, 2*x12 - 2*x02
C1, D1 = y01 - 2*y11 + y21, 2*y11 - 2*y01
C2, D2 = y02 - 2*y12 + y22, 2*y12 - 2*y02

a = 4*(A1**2*C2**2 - 2*A1*A2*C1*C2 + A2**2*C1**2)*A2**2
b = -8*(A1*C2 - A2*C1)*A2**2*(A2*D1 - B1*C2)
c = -4*(A1*(A2*(2*C2*(y01 - y02) + D2**2) - (B2*D2 + 2*C2*(x01 - x02))*C2) - A2**2*(2*C1*(y01 - y02) + D1**2) + A2*(2*B1*C2*D1 - (B2*D2 - 2*C2*(x01 - x02))*C1) - (B1**2*C2 - B2**2*C1)*C2)*A2**2
d = 4*A2**2*(2*A2**2*D1*(y01 - y02) - A2*(B1*(2*C2*(y01 - y02) + D2**2) - (B2*D2 - 2*C2*(x01 - x02))*D1) + (B1*(B2*D2 + 2*C2*(x01 - x02)) - B2**2*D1)*C2)
e = 4*A2**2*(A2**2*(y01**2 - 2*y01*y02 + y02**2) + A2*(B2*D2*(y01 - y02) - (2*C2*(y01 - y02) + D2**2)*(x01 - x02)) - (B2**2*(y01 - y02) - B2*D2*(x01 - x02) - C2*(x01**2 - 2*x01*x02 + x02**2))                                                                                                                                       
roots = np.roots([a,b,c,d,e])
nodes = [seg1.start,seg1.control,seg1.end,                                                                                                                                                                                             
          seg2.start,seg2.control,seg2.end]

nodes = nodes + [seg1.point(t) for t in roots]

disvg(path,nodes=nodes)
# if any(abs(root) <= 1 and root.imag == 0 for root in roots):
#     disvg(path,nodes=nodes)

In [13]:
print("""
4*(a1^(2)*c2^(2)-2*a1*a2*c1*c2+a2^(2)*c1^(2))*a2^(2),−8*(a1*c2-a2*c1)*a2^(2)*(a2*d1-b1*c2),−4*(a1*(a2*(2*c2*(y01-y02)+d2^(2))-(b2*d2+2*c2*(x01-x02))*c2)-a2^(2)*(2*c1*(y01-y02)+d1^(2))+a2*(2*b1*c2*d1-(b2*d2-2*c2*(x01-x02))*c1)-(b1^(2)*c2-b2^(2)*c1)*c2)*a2^(2),4*a2^(2)*(2*a2^(2)*d1*(y01-y02)-a2*(b1*(2*c2*(y01-y02)+d2^(2))-(b2*d2-2*c2*(x01-x02))*d1)+(b1*(b2*d2+2*c2*(x01-x02))-b2^(2)*d1)*c2),4*a2^(2)*(a2^(2)*(y01^(2)-2*y01*y02+y02^(2))+a2*(b2*d2*(y01-y02)-(2*c2*(y01-y02)+d2^(2))*(x01-x02))-(b2^(2)*(y01-y02)-b2*d2*(x01-x02)-c2*(x01^(2)-2*x01*x02+x02^(2)))*c2)
""".replace('+',' + ').replace('-',' - ').replace(',','\n'))


4*(a1^(2)*c2^(2) - 2*a1*a2*c1*c2 + a2^(2)*c1^(2))*a2^(2)
−8*(a1*c2 - a2*c1)*a2^(2)*(a2*d1 - b1*c2)
−4*(a1*(a2*(2*c2*(y01 - y02) + d2^(2)) - (b2*d2 + 2*c2*(x01 - x02))*c2) - a2^(2)*(2*c1*(y01 - y02) + d1^(2)) + a2*(2*b1*c2*d1 - (b2*d2 - 2*c2*(x01 - x02))*c1) - (b1^(2)*c2 - b2^(2)*c1)*c2)*a2^(2)
4*a2^(2)*(2*a2^(2)*d1*(y01 - y02) - a2*(b1*(2*c2*(y01 - y02) + d2^(2)) - (b2*d2 - 2*c2*(x01 - x02))*d1) + (b1*(b2*d2 + 2*c2*(x01 - x02)) - b2^(2)*d1)*c2)
4*a2^(2)*(a2^(2)*(y01^(2) - 2*y01*y02 + y02^(2)) + a2*(b2*d2*(y01 - y02) - (2*c2*(y01 - y02) + d2^(2))*(x01 - x02)) - (b2^(2)*(y01 - y02) - b2*d2*(x01 - x02) - c2*(x01^(2) - 2*x01*x02 + x02^(2)))*c2)



In [14]:
eq = """
4*(a1^(2)*c2^(2) - 2*a1*a2*c1*c2 + a2^(2)*c1^(2))*a2^(2)
-8*(a1*c2 - a2*c1)*a2^(2)*(a2*d1 - b1*c2)
-4*(a1*(a2*(2*c2*(y01 - y02) + d2^(2)) - (b2*d2 + 2*c2*(x01 - x02))*c2) - a2^(2)*(2*c1*(y01 - y02) + d1^(2)) + a2*(2*b1*c2*d1 - (b2*d2 - 2*c2*(x01 - x02))*c1) - (b1^(2)*c2 - b2^(2)*c1)*c2)*a2^(2)
4*a2^(2)*(2*a2^(2)*d1*(y01 - y02) - a2*(b1*(2*c2*(y01 - y02) + d2^(2)) - (b2*d2 - 2*c2*(x01 - x02))*d1) + (b1*(b2*d2 + 2*c2*(x01 - x02)) - b2^(2)*d1)*c2)
4*a2^(2)*(a2^(2)*(y01^(2) - 2*y01*y02 + y02^(2)) + a2*(b2*d2*(y01 - y02) - (2*c2*(y01 - y02) + d2^(2))*(x01 - x02)) - (b2^(2)*(y01 - y02) - b2*d2*(x01 - x02) - c2*(x01^(2) - 2*x01*x02 + x02^(2)))*c2)
""".replace('₀','0').replace('₁','1').replace('₂','2').replace('^','**').replace('o','0')
eq=eq.replace('a','A').replace('b','B').replace('c','C').replace('d','D').replace('(2)','2').replace('(3)','3').replace('(4)','4')
print(eq)


4*(A1**2*C2**2 - 2*A1*A2*C1*C2 + A2**2*C1**2)*A2**2
-8*(A1*C2 - A2*C1)*A2**2*(A2*D1 - B1*C2)
-4*(A1*(A2*(2*C2*(y01 - y02) + D2**2) - (B2*D2 + 2*C2*(x01 - x02))*C2) - A2**2*(2*C1*(y01 - y02) + D1**2) + A2*(2*B1*C2*D1 - (B2*D2 - 2*C2*(x01 - x02))*C1) - (B1**2*C2 - B2**2*C1)*C2)*A2**2
4*A2**2*(2*A2**2*D1*(y01 - y02) - A2*(B1*(2*C2*(y01 - y02) + D2**2) - (B2*D2 - 2*C2*(x01 - x02))*D1) + (B1*(B2*D2 + 2*C2*(x01 - x02)) - B2**2*D1)*C2)
4*A2**2*(A2**2*(y01**2 - 2*y01*y02 + y02**2) + A2*(B2*D2*(y01 - y02) - (2*C2*(y01 - y02) + D2**2)*(x01 - x02)) - (B2**2*(y01 - y02) - B2*D2*(x01 - x02) - C2*(x01**2 - 2*x01*x02 + x02**2))*C2)



In [ ]:
se={1,2,3,3,4}
se.update({5,2})
se

In [ ]:
for s in {-1,1}:
    print(s)